In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# Aggregates using window function

In [16]:
# sqlite version

pd.read_sql_query(sql='''
SELECT id, account_id, standard_qty,
strftime('%Y-%m', occurred_at) month,
DENSE_RANK() OVER (PARTITION BY account_id ORDER BY strftime('%Y-%m', occurred_at)) dense_rank,
SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY strftime('%Y-%m', occurred_at)) sum_standard_qty,
COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY strftime('%Y-%m', occurred_at)) count_standard_qty,
AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY strftime('%Y-%m', occurred_at)) avg_standard_qty,
MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY strftime('%Y-%m', occurred_at)) min_standard_qty,
MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY strftime('%Y-%m', occurred_at)) max_standard_qty
FROM orders;
''', con=conn)

,id,account_id,standard_qty,month,dense_rank,sum_standard_qty,count_standard_qty,avg_standard_qty,min_standard_qty,max_standard_qty
0,1,1001,123,2015-10,1,123,1,123.000000,123,123
1,2,1001,190,2015-11,2,819,3,273.000000,123,506
2,4307,1001,506,2015-11,2,819,3,273.000000,123,506
3,3,1001,85,2015-12,3,1430,5,286.000000,85,526
4,4308,1001,526,2015-12,3,1430,5,286.000000,85,526
...,...,...,...,...,...,...,...,...,...,...
6907,4304,4501,159,2016-10,6,759,9,84.333333,5,180
6908,4305,4501,6,2016-11,7,828,11,75.272727,5,180
6909,6911,4501,63,2016-11,7,828,11,75.272727,5,180
6910,4306,4501,126,2016-12,8,1015,13,78.076923,5,180
